# Wrapper notebook for submitting the RBP maps script to TSCC

In [3]:
import pandas as pd
import os
import json
import yaml
import numpy as np
import glob
# import rethinkdb as r
from collections import defaultdict
from qtools import Submitter
from encode import manifest_helpers as m

from tqdm import tnrange, tqdm_notebook
pd.set_option("display.max_colwidth", 10000)
pd.set_option('display.max_columns', 500)

# Define programs

In [42]:
density_runner = '/home/bay001/projects/codebase/rbp-maps/maps/plot_map.py'
peak_runner = '/home/bay001/projects/codebase/rbp-maps/maps/plot_map.py'

annotation_dir = '/projects/ps-yeolab3/bay001/maps/current_annotations/se/'
control_annotation_dir = '/projects/ps-yeolab3/bay001/maps/current_annotations/erics_controls/nonredundant_renamed/se/'

params = {
    'whole_read':{
        'output_dir' : '/projects/ps-yeolab3/bay001/maps/current/se',
        'clip_manifest' : '/projects/ps-yeolab3/bay001/reference_data/ENCODE/ENCODE_FINAL_ANNOTATIONS.uidsonly.txt.manifest.txt',
        'prefix' : 'whole_read',
        'website_manifest' : '/home/bay001/projects/maps_20160420/permanent_data/ALLDATASETS_submittedonly.website.wholeread.txt'
    },
    '5p':{
        'output_dir' : '/projects/ps-yeolab3/bay001/maps/current/se_5p/',
        'clip_manifest' : '/home/bay001/projects/maps_20160420/permanent_data/ALLDATASETS_submittedonly.5p.txt',
        'prefix' : '5p',
        'website_manifest' : '/home/bay001/projects/maps_20160420/permanent_data/ALLDATASETS_submittedonly.website.5p.txt'
    },
    'peak_bb':{
        'output_dir' : '/projects/ps-yeolab3/bay001/maps/current/se_peak/',
        'clip_manifest' : '/projects/ps-yeolab3/bay001/reference_data/ENCODE/ENCODE_FINAL_ANNOTATIONS.uidsonly.txt.manifest.txt',
        'prefix' : 'peak',
        'peak_dir' : '/projects/ps-yeolab3/bay001/maps/current_annotations/se_peak_bigbeds/'
    },
    'idr_bb':{
        'output_dir':'/home/bay001/scratch/maps/se/idr/',
        # 'output_dir' : '/projects/ps-yeolab3/bay001/maps/current/se_idr_peak/',
        'clip_manifest' : '/projects/ps-yeolab3/bay001/reference_data/ENCODE/ENCODE_FINAL_ANNOTATIONS.uidsonly.txt.manifest.txt',
        'prefix' : 'peak',
        'peak_dir' : '/projects/ps-yeolab3/bay001/maps/current_annotations/se_peak_idr_bigbeds'
    },
    'whole_read_xcompare':{
        'output_dir' : '/home/bay001/projects/brian_rbpmaps_20180202/temporary_data/se_xcompare/',
        # 'output_dir' : '/projects/ps-yeolab3/bay001/maps/current/se_xcompare',
        'clip_manifest' : '/projects/ps-yeolab3/bay001/reference_data/ENCODE/ENCODE_FINAL_ANNOTATIONS.uidsonly.txt.manifest.txt',
        'prefix' : 'whole_read_xcompare',
        'website_manifest' : '/home/bay001/projects/maps_20160420/permanent_data/ALLDATASETS_submittedonly.website.wholeread.txt'
    },
}

# Define manifests, directories, etc.

In [5]:
current_date = '8-20-2018'
bash_scripts_dir = '/projects/ps-yeolab3/bay001/maps/bash_scripts/{}'.format(current_date)
! mkdir $bash_scripts_dir

# Define helper scripts for accessing/joining RNASEQ and eCLIP annotations

In [6]:
read_type = 'whole_read'

clip_df = pd.read_table(params[read_type]['clip_manifest'], sep='\t')

master_table = pd.read_table(
    '/projects/ps-yeolab3/encode/rnaseq/eCLIP_finalstatus_20180406_ENCODE_combined_RNASEQ.tsv'
)
del master_table['eCLIP_Antibody_Lot_#']  # there is a NaN value in here that's screwing things up. Don't care about antibody anyway, so delete
merged = pd.merge(
    clip_df,
    master_table,
    how='right',
    left_on=['uID'],
    right_on=['eCLIP_Final_internal_accession']
)
merged.dropna(subset=['eCLIP_Final_internal_accession','SE_jxc_file'], inplace=True)
print(merged.shape)

(203, 43)


In [36]:
merged

,uID,RBP,Cell line,CLIP_rep1,CLIP_rep2,INPUT,eCLIP_uID,eCLIP_Official_Gene_Symbol,eCLIP_Final_internal_accession,eCLIP_Submission_status,eCLIP_Antibody_Cat_#,eCLIP_Cell_Line,RNASEQ_Experiment_ID,RNASEQ_RBP,RNASEQ_Official_RBP,RNASEQ_Cell_line,RNASEQ_Control_Experiment_ID,CONTROL_RBP,CONTROL_Cell_line,RNASEQ_Replicate_rep1,RNASEQ_FASTQ_R1_rep1,RNASEQ_FASTQ_R2_rep1,RNASEQ_BAM_rep1,RNASEQ_TSV_rep1,Rep_rep1,CONTROL_Replicate_rep1,CONTROL_FASTQ_R1_rep1,CONTROL_FASTQ_R2_rep1,CONTROL_BAM_rep1,CONTROL_TSV_rep1,RNASEQ_Replicate_rep2,RNASEQ_FASTQ_R1_rep2,RNASEQ_FASTQ_R2_rep2,RNASEQ_BAM_rep2,RNASEQ_TSV_rep2,Rep_rep2,CONTROL_Replicate_rep2,CONTROL_FASTQ_R1_rep2,CONTROL_FASTQ_R2_rep2,CONTROL_BAM_rep2,CONTROL_TSV_rep2,SE_jxc_file,RNASEQ_DESeq2
0,203,HNRNPC,HepG2,/projects/ps-yeolab3/encode/analysis/encode_master/271_01_HNRNPC.merged.r2.bam,/projects/ps-yeolab3/encode/analysis/encode_master/203_02_HNRNPC.merged.r2.bam,/projects/ps-yeolab3/encode/analysis/encode_master/271_INPUT_ATTACTCG-GGCTCTGA_L002_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.r2.bam,203.0,HNRNPC,203,DONE - 2 reps submitted 9/22/15,RN052PW,HepG2,ENCSR052IYH,HNRNPC,HNRNPC,HepG2,ENCSR305XWT,non-target,HepG2,HNRNPC-BGHLV20-33_17722,ENCFF682EBX,ENCFF801LSJ,ENCFF531AUK,ENCFF132FTR,1.0,non-target-BGHLV20-1_17706,ENCFF711JSH,ENCFF941XAQ,ENCFF460AAE,ENCFF024EGT,HNRNPC-BGHLV20-34_17723,ENCFF078KLK,ENCFF970UVV,ENCFF031QZG,ENCFF064QUI,2.0,non-target-BGHLV20-2_17707,ENCFF701KJC,ENCFF603JIF,ENCFF058QDT,ENCFF939PSG,HNRNPC-BGHLV20-HepG2.set20.SE.MATS.JunctionCountOnly.txt,HepG2_HNRNPC_BGHLV20_DESeq_output.txt
1,204,RBFOX2,HepG2,/projects/ps-yeolab3/encode/analysis/encode_master/204_01_RBFOX2.merged.r2.bam,/projects/ps-yeolab3/encode/analysis/encode_master/204_02_RBFOX2.merged.r2.bam,/projects/ps-yeolab3/encode/analysis/encode_master/RBFOX2-204-INPUT_S2_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.r2.bam,204.0,RBFOX2,204,DONE - 2 reps submitted 6/28/15,A300-864A,HepG2,ENCSR767LLP,RBFOX2,RBFOX2,HepG2,ENCSR104ABF,non-target,HepG2,RBFOX2_BGHLV26_35,ENCFF201PEI,ENCFF508QNI,ENCFF347ERZ,ENCFF222JWL,1.0,non-target_BGHLV26-1,ENCFF291QQH,ENCFF602GIQ,ENCFF988VWE,ENCFF653XRX,RBFOX2_BGHLV26_36,ENCFF040CTS,ENCFF435HLB,ENCFF946VPZ,ENCFF042VXF,2.0,non-target_BGHLV26-2,ENCFF503VRZ,ENCFF105YHI,ENCFF893QHC,ENCFF401ECA,RBFOX2-BGHLV26-HepG2.set26.SE.MATS.JunctionCountOnly.txt,HepG2_RBFOX2_BGHLV26_DESeq_output.txt
2,205,IGF2BP1,HepG2,/projects/ps-yeolab3/encode/analysis/encode_master/205_01_IGF2BP1.merged.r2.bam,/projects/ps-yeolab3/encode/analysis/encode_master/205_02_IGF2BP1.merged.r2.bam,/projects/ps-yeolab3/encode/analysis/encode_master/IGF2BP1-205-INPUT_S4_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.r2.bam,205.0,IGF2BP1,205,DONE - 2 reps submitted 6/28/15,RN007P,HepG2,ENCSR268XHM,IGF2BP1,IGF2BP1,HepG2,ENCSR105NML,non-target,HepG2,IGF2BP1_BGHcLV03-3,ENCFF177UID,ENCFF051UMI,ENCFF326QKH,ENCFF750JCA,1.0,NT_BGHcLV03-1,ENCFF255GEL,ENCFF930LDL,ENCFF534AWV,ENCFF272JSE,IGF2BP1_BGHcLV03-4,ENCFF641TDC,ENCFF712MFS,ENCFF384APO,ENCFF409LBZ,2.0,NT_BGHcLV03-2,ENCFF582GSM,ENCFF299GUW,ENCFF134QUL,ENCFF816DSO,IGF2BP1-BGHcLV03-HepG2.BGHcLV03.SE.MATS.JunctionCountOnly.txt,HepG2_IGF2BP1_BGHcLV03_DESeq_output.txt
3,206,HNRNPK,HepG2,/projects/ps-yeolab3/encode/analysis/encode_master/206_01_HNRNPK.merged.r2.bam,/projects/ps-yeolab3/encode/analysis/encode_master/206_02_HNRNPK.merged.r2.bam,/projects/ps-yeolab3/encode/analysis/encode_master/206-INPUT_S10_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.r2.bam,206.0,HNRNPK,206,DONE - 2 reps submitted 9/22/15,RN019P,HepG2,ENCSR853ZJS,HNRNPK,HNRNPK,HepG2,ENCSR237YZT,non-target,HepG2,HNRNPK-BGHLV12-37,ENCFF106HXI,ENCFF763SDA,ENCFF033EZX,ENCFF238QRM,1.0,NT-BGHLV12-01,ENCFF801YDB,ENCFF794CUL,ENCFF857QSU,ENCFF427UKK,HNRNPK-BGHLV12-38,ENCFF462KWR,ENCFF949CWD,ENCFF589WIS,ENCFF536ZWN,2.0,NT-BGHLV12-02,ENCFF619LUS,ENCFF200OMG,ENCFF052HTH,ENCFF025LXR,HNRNPK-BGHLV12-HepG2.set12.SE.MATS.JunctionCountOnly.txt,HepG2_HNRNPK_BGHLV12_DESeq_output.txt
4,209,SRSF7,HepG2,/projec

# Plot SE Density

In [8]:
def get_clip_file_from_uid(uid, df=merged):
    df = df[df['uID']==uid]
    return df['CLIP_rep1'].values[0], \
            df['CLIP_rep2'].values[0], \
            df['INPUT'].values[0], \
            df['eCLIP_Official_Gene_Symbol'].values[0], \
            df['eCLIP_Cell_Line'].values[0], \
            df['SE_jxc_file'].values[0]

r1, r2, i, rbp, cell, jxc_se  = get_clip_file_from_uid('204')

def get_annotations_from_jxc_se(jxc, jxc_dir=annotation_dir):
    """ jxc contains the basename of the junction counts file """
    orig_file = os.path.join(jxc_dir, jxc)
    positive = os.path.splitext(orig_file)[0] + ".positive.nr.txt"
    positive = positive.replace('.SE.MATS.JunctionCountOnly.positive.nr.txt','-included-upon-knockdown')
    negative = os.path.splitext(orig_file)[0] + ".negative.nr.txt"
    negative = negative.replace('.SE.MATS.JunctionCountOnly.negative.nr.txt','-excluded-upon-knockdown')
    assert os.path.exists(orig_file)
    if not os.path.exists(positive):
        positive = None
    if not os.path.exists(negative):
        negative = None
    return positive, negative

get_annotations_from_jxc_se(jxc_se)

('/projects/ps-yeolab3/bay001/maps/current_annotations/se/RBFOX2-BGHLV26-HepG2.set26-included-upon-knockdown',
 '/projects/ps-yeolab3/bay001/maps/current_annotations/se/RBFOX2-BGHLV26-HepG2.set26-excluded-upon-knockdown')

In [10]:
img_extension = ['svg']
pos_splicing_suffix = '-included-upon-knockdown'
neg_splicing_suffix = '-excluded-upon-knockdown'

if not os.path.exists(bash_scripts_dir):
    ! mkdir $bash_scripts_dir

read_type = 'whole_read'
normalization_levels = [4]

### Force override of maps
force = False

### DEFINE BACKGROUNDS (THESE ARE STATIC AND DON'T CHANGE) ###
k562_background_ce = os.path.join(control_annotation_dir, 'K562_constitutive_exons')
k562_background_nse_all = os.path.join(control_annotation_dir, 'K562_native_cassette_exons_all')
k562_background_nse_inc = os.path.join(control_annotation_dir, 'K562_natively_included_cassette_exons')
k562_background_nse_exc = os.path.join(control_annotation_dir, 'K562_natively_excluded_cassette_exons')
k562_background_nse_avg = os.path.join(control_annotation_dir, 'K562_native_cassette_exons_avg')

hepg2_background_ce = os.path.join(control_annotation_dir, 'HepG2_constitutive_exons')
hepg2_background_nse_all = os.path.join(control_annotation_dir, 'HepG2_native_cassette_exons_all')
hepg2_background_nse_inc = os.path.join(control_annotation_dir, 'HepG2_natively_included_cassette_exons')
hepg2_background_nse_exc = os.path.join(control_annotation_dir, 'HepG2_natively_excluded_cassette_exons')
hepg2_background_nse_avg = os.path.join(control_annotation_dir, 'HepG2_native_cassette_exons_avg')

# no_rnaseq = [] # uIDs for which we don't have rna seq expt ids for
no_rnaseq_yet = [] # uIDs for which we have an expt id, but haven't downloaded the data yet (or there were no significant splice events)
cmds = []

for normalization_level in normalization_levels:
    for ext in img_extension:
        for uid in merged['uID']:
            r1, r2, i, rbp, cell, jxc_se = get_clip_file_from_uid(uid)

            if cell == 'K562':
                background1 = k562_background_ce
                background2 = k562_background_nse_all
                background3 = k562_background_nse_inc
                background4 = k562_background_nse_exc
                background5 = k562_background_nse_avg
            elif cell == 'HepG2':
                background1 = hepg2_background_ce
                background2 = hepg2_background_nse_all
                background3 = hepg2_background_nse_inc
                background4 = hepg2_background_nse_exc
                background5 = hepg2_background_nse_avg
            else:
                print(cell)


            ##### get the positive and negative associated annotations using this prefix #####
            positive, negative = get_annotations_from_jxc_se(
                jxc_se
            )
            ### we HAVE to have at both positive and negative annotations to plot ###
            if(positive == None or negative == None):
                no_rnaseq_yet.append(uid)
            else:
                ### uses RBP name to ensure positive and negative annotations are being pulled ###
                pos_prefix = os.path.basename(positive).split('-')[0]
                neg_prefix = os.path.basename(negative).split('-')[0]
                if not (pos_prefix in rbp and neg_prefix in rbp):
                    print(
                        'warning, these dont match: {}, {}, {}'.format(
                            rbp, 
                            os.path.basename(positive),
                            os.path.basename(negative)
                        )
                    )
                ### Foreach replicate, build teh command used to call the python script.
                for r in [r1, r2]:
                    name = os.path.basename(r).replace('.bam','.{}.{}'.format(normalization_level, ext))
                    output_filename = os.path.join(
                        params[read_type]['output_dir'],
                        name
                    )

                    # Build the cmd line
                    cmd = "python " + density_runner
                    cmd = cmd + " --event {}".format('se')
                    cmd = cmd + " --ipbam {}".format(r)
                    cmd = cmd + " --inputbam {}".format(i)
                    cmd = cmd + " --output {}".format(output_filename)
                    cmd = cmd + " --annotations {} {} {} {} {} {} {}".format(
                        positive, negative, background1, background2, background3, background4, background5
                    )
                    cmd = cmd + " --annotation_type {} {} {} {} {} {} {}".format(
                        'rmats', 'rmats', 'eric', 'eric', 'eric', 'eric', 'eric' 
                    )
                    # cmd = cmd + " --chrom_sizes {}".format(chrom_sizes)
                    cmd = cmd + " --testnum {} {}".format(0, 1)
                    cmd = cmd + " --bgnum {}".format(3) # test against native SE
                    cmd = cmd + " --normalization_level {}".format(normalization_level)
                    cmd = cmd + " --sigtest permutation"
                    if not os.path.exists(output_filename) or force == True:
                        cmds.append(cmd)

bash_script_sh = '/projects/ps-yeolab3/bay001/maps/bash_scripts/{}/{}-SE_NR_{}.{}.sh'.format(
    current_date, 
    params[read_type]['prefix'], 
    ext,
    normalization_level
)

Submitter(
    cmds, 
    "{}-SE_NR_{}".format(params[read_type]['prefix'], ext), 
    sh=bash_script_sh,
    submit=False,
    array=True,
    walltime='3:00:00',
    queue='home-yeo'
)

### Print any missing/unavailable annotations to check over ###
print("uIDs for which we have an RNASEQ ID, but we don't have the splicing data yet: {}".format(
        len(no_rnaseq_yet))
     )

warning, these dont match: DROSHA, RNASEN50-BGKLV28-K562.set28-included-upon-knockdown, RNASEN50-BGKLV28-K562.set28-excluded-upon-knockdown
warning, these dont match: YBX3, CSDA-BGKLV24-K562.set24-included-upon-knockdown, CSDA-BGKLV24-K562.set24-excluded-upon-knockdown
warning, these dont match: YBX3, CSDA-BGHLV20-HepG2.set20-included-upon-knockdown, CSDA-BGHLV20-HepG2.set20-excluded-upon-knockdown
uIDs for which we have an RNASEQ ID, but we don't have the splicing data yet: 10


Writing 0 tasks as an array-job.
Wrote commands to /projects/ps-yeolab3/bay001/maps/bash_scripts/8-20-2018/whole_read-SE_NR_svg.4.sh.


# Ensure we have made all of the maps we need for integrated paper.

In [12]:
ext = 'svg'
for uid in merged['uID']:
    r1, r2, i, rbp, cell, jxc_se = get_clip_file_from_uid(uid)
    positive, negative = get_annotations_from_jxc_se(
        jxc_se
    )
    if(positive == None or negative == None):
        pass
    else:
        pdf = pd.read_table(positive)
        ndf = pd.read_table(negative)

        if(pdf.shape[0] >= 100 and ndf.shape[0] >= 100):
            for r in [r1, r2]:
                name = os.path.basename(r).replace('.bam','.{}.{}'.format(normalization_level, ext))
                means = glob.glob(
                    os.path.join(
                        params[read_type]['output_dir'],
                        os.path.basename(r).replace('.bam','*.means.txt')
                    )
                )
                output_filename = os.path.join(
                    params[read_type]['output_dir'],
                    name
                )
                if not os.path.exists(output_filename):
                    print("{} {} doesnt exist".format(output_filename, jxc_se))
                if len(means) != 7:
                    print("missing means (found {})".format(means))

# Plot SE Peaks

In [17]:
key = 'peak_bb'

img_extensions = ['svg'] #,'svg']
clip_df = pd.read_table(params[key]['clip_manifest'])

no_rnaseq = [] # uIDs for which we don't have rna seq expt ids for
no_rnaseq_yet = [] # uIDs for which we have an expt id, but haven't downloaded the data yet
cmds = []
force = True

### DEFINE BACKGROUNDS (THESE ARE STATIC AND DON'T CHANGE) ###
k562_background_ce = os.path.join(control_annotation_dir, 'K562_constitutive_exons')
k562_background_nse_all = os.path.join(control_annotation_dir, 'K562_native_cassette_exons_all')
k562_background_nse_inc = os.path.join(control_annotation_dir, 'K562_natively_included_cassette_exons')
k562_background_nse_exc = os.path.join(control_annotation_dir, 'K562_natively_excluded_cassette_exons')
k562_background_nse_avg = os.path.join(control_annotation_dir, 'K562_native_cassette_exons_avg')

hepg2_background_ce = os.path.join(control_annotation_dir, 'HepG2_constitutive_exons')
hepg2_background_nse_all = os.path.join(control_annotation_dir, 'HepG2_native_cassette_exons_all')
hepg2_background_nse_inc = os.path.join(control_annotation_dir, 'HepG2_natively_included_cassette_exons')
hepg2_background_nse_exc = os.path.join(control_annotation_dir, 'HepG2_natively_excluded_cassette_exons')
hepg2_background_nse_avg = os.path.join(control_annotation_dir, 'HepG2_native_cassette_exons_avg')


for ext in img_extensions:
    for uid in merged['uID']:
        peak_files = glob.glob(os.path.join(
            params[key]['peak_dir'], 
            '{}_0*.basedon_{}_0*.peaks.l2inputnormnew.bed.compressed.bed.p3f3.bed.sorted.bed.bb'.format(
                uid, uid
            )
        ))
        assert len(peak_files) == 2
        
        for peak in peak_files:
            output_filename = os.path.join(
                params[key]['output_dir'],
                os.path.basename(peak).replace('.bb','.bb.{}'.format(ext))
            )
            r1, r2, i, rbp, cell, jxc_se = get_clip_file_from_uid(uid)


            if cell == 'K562':
                background1 = k562_background_ce
                background2 = k562_background_nse_all
                background3 = k562_background_nse_inc
                background4 = k562_background_nse_exc
                background5 = k562_background_nse_avg
            elif cell == 'HepG2':
                background1 = hepg2_background_ce
                background2 = hepg2_background_nse_all
                background3 = hepg2_background_nse_inc
                background4 = hepg2_background_nse_exc
                background5 = hepg2_background_nse_avg
            else:
                print(cell)

            ##### get the positive and negative associated annotations using this prefix #####
            positive, negative = get_annotations_from_jxc_se(
                jxc_se
            )
            if uid == '560':
                print(positive, negative)
            if(positive == None or negative == None):
                no_rnaseq_yet.append(uid)
            else:
                # Build the cmd line
                cmd = "python " + peak_runner
                cmd = cmd + " --event {}".format('se')
                cmd = cmd + " --peak {}".format(peak)
                cmd = cmd + " --output {}".format(output_filename)
                cmd = cmd + " --annotations {} {} {} {} {} {} {}".format(
                    positive, negative, background1, background2, background3, background4, background5
                )
                cmd = cmd + " --annotation_type {} {} {} {} {} {} {}".format(
                    'rmats', 'rmats', 'eric', 'eric', 'eric', 'eric', 'eric' 
                )
                # cmd = cmd + " --chrom_sizes {}".format(chrom_sizes)
                cmd = cmd + " --testnum {} {}".format(0, 1)
                cmd = cmd + " --bgnum {}".format(3) # test against native SE
                cmd = cmd + " --normalization_level {}".format(0)
                cmd = cmd + " --sigtest {}".format("fisher")
                if not os.path.exists(output_filename) or force == True:
                    cmds.append(cmd)

bash_script_sh = '/projects/ps-yeolab3/bay001/maps/bash_scripts/{}/SE_PEAK_PNGS.sh'.format(current_date)

Submitter(
    cmds, 
    "SE_PEAK_PNGS", 
    sh=bash_script_sh,
    submit=False,
    array=True,
    walltime='0:20:00',
    queue='home-yeo'
)

with open(bash_script_sh.replace('.sh','.missing.txt'), 'w') as o:
    for no in no_rnaseq:
        o.write(
            '{}\t{}\n'.format(
                m.get_clip_file_from_uid(clip_df, no)[3],
                m.get_clip_file_from_uid(clip_df, no)[4],
            )
        )
    print("\n\nNO SUFFICIENT POSITIVE OR NEGATIVE SIGNIFICANT ANNOTATIONS:")
    for no in no_rnaseq_yet:
        print(no)

('/projects/ps-yeolab3/bay001/maps/current_annotations/se/HNRNPL-BGKLV25-K562.set25-included-upon-knockdown', '/projects/ps-yeolab3/bay001/maps/current_annotations/se/HNRNPL-BGKLV25-K562.set25-excluded-upon-knockdown')
('/projects/ps-yeolab3/bay001/maps/current_annotations/se/HNRNPL-BGKLV25-K562.set25-included-upon-knockdown', '/projects/ps-yeolab3/bay001/maps/current_annotations/se/HNRNPL-BGKLV25-K562.set25-excluded-upon-knockdown')


NO SUFFICIENT POSITIVE OR NEGATIVE SIGNIFICANT ANNOTATIONS:
256
256
275
275
279
279
444
444
460
460
586
586
645
645
681
681
697
697
722reseq2
722reseq2


Writing 386 tasks as an array-job.
Wrote commands to /projects/ps-yeolab3/bay001/maps/bash_scripts/7-25-2018/SE_PEAK_PNGS.sh.


# IDR peak splice maps

In [7]:
key = 'idr_bb'

img_extensions = ['svg'] #,'svg']
clip_df = pd.read_table(params[key]['clip_manifest'])

no_rnaseq = [] # uIDs for which we don't have rna seq expt ids for
no_rnaseq_yet = [] # uIDs for which we have an expt id, but haven't downloaded the data yet
cmds = []
force = True

### DEFINE BACKGROUNDS (THESE ARE STATIC AND DON'T CHANGE) ###
k562_background_ce = os.path.join(control_annotation_dir, 'K562_constitutive_exons')
k562_background_nse_all = os.path.join(control_annotation_dir, 'K562_native_cassette_exons_all')
k562_background_nse_inc = os.path.join(control_annotation_dir, 'K562_natively_included_cassette_exons')
k562_background_nse_exc = os.path.join(control_annotation_dir, 'K562_natively_excluded_cassette_exons')
k562_background_nse_avg = os.path.join(control_annotation_dir, 'K562_native_cassette_exons_avg')

hepg2_background_ce = os.path.join(control_annotation_dir, 'HepG2_constitutive_exons')
hepg2_background_nse_all = os.path.join(control_annotation_dir, 'HepG2_native_cassette_exons_all')
hepg2_background_nse_inc = os.path.join(control_annotation_dir, 'HepG2_natively_included_cassette_exons')
hepg2_background_nse_exc = os.path.join(control_annotation_dir, 'HepG2_natively_excluded_cassette_exons')
hepg2_background_nse_avg = os.path.join(control_annotation_dir, 'HepG2_native_cassette_exons_avg')


for ext in img_extensions:
    for uid in merged['uID']:
        peak_files = glob.glob(os.path.join(
            params[key]['peak_dir'], 
            '{}.01v02.IDR.out.0102merged.bed.blacklist_removed.bed.p0f0.bed.sorted.bed.bb'.format(
                uid
            )
        ))

        assert len(peak_files) == 1
        
        for peak in peak_files:
            output_filename = os.path.join(
                params[key]['output_dir'],
                os.path.basename(peak).replace('.bb','.bb.{}'.format(ext))
            )
            r1, r2, i, rbp, cell, jxc_se = get_clip_file_from_uid(uid)


            if cell == 'K562':
                background1 = k562_background_ce
                background2 = k562_background_nse_all
                background3 = k562_background_nse_inc
                background4 = k562_background_nse_exc
                background5 = k562_background_nse_avg
            elif cell == 'HepG2':
                background1 = hepg2_background_ce
                background2 = hepg2_background_nse_all
                background3 = hepg2_background_nse_inc
                background4 = hepg2_background_nse_exc
                background5 = hepg2_background_nse_avg
            else:
                print(cell)

            ##### get the positive and negative associated annotations using this prefix #####
            positive, negative = get_annotations_from_jxc_se(
                jxc_se
            )
            if uid == '678':
                print(positive, negative)
            if(positive == None or negative == None):
                no_rnaseq_yet.append(uid)
            else:
                # Build the cmd line
                cmd = "python " + peak_runner
                cmd = cmd + " --event {}".format('se')
                cmd = cmd + " --peak {}".format(peak)
                cmd = cmd + " --output {}".format(output_filename)
                cmd = cmd + " --annotations {} {} {} {} {} {} {}".format(
                    positive, negative, background1, background2, background3, background4, background5
                )
                cmd = cmd + " --annotation_type {} {} {} {} {} {} {}".format(
                    'rmats', 'rmats', 'eric', 'eric', 'eric', 'eric', 'eric' 
                )
                # cmd = cmd + " --chrom_sizes {}".format(chrom_sizes)
                cmd = cmd + " --testnum {} {}".format(0, 1)
                cmd = cmd + " --bgnum {}".format(3) # test against native SE
                cmd = cmd + " --normalization_level {}".format(0)
                cmd = cmd + " --sigtest {}".format("fisher")
                cmd = cmd + " --confidence 1"
                if not os.path.exists(output_filename) or force == True:
                    cmds.append(cmd)

bash_script_sh = '/projects/ps-yeolab3/bay001/maps/bash_scripts/{}/SE_IDR_PNGS.sh'.format(current_date)

Submitter(
    cmds, 
    "SE_IDR_SVGS", 
    sh=bash_script_sh,
    submit=False,
    array=True,
    walltime='0:20:00',
    queue='home-yeo'
)

with open(bash_script_sh.replace('.sh','.missing.txt'), 'w') as o:
    for no in no_rnaseq:
        o.write(
            '{}\t{}\n'.format(
                m.get_clip_file_from_uid(clip_df, no)[3],
                m.get_clip_file_from_uid(clip_df, no)[4],
            )
        )
    print("\n\nNO SUFFICIENT POSITIVE OR NEGATIVE SIGNIFICANT ANNOTATIONS:")
    for no in no_rnaseq_yet:
        print(no)

('/projects/ps-yeolab3/bay001/maps/current_annotations/se/HNRNPL-BGHLV23-HepG2.set23-included-upon-knockdown', '/projects/ps-yeolab3/bay001/maps/current_annotations/se/HNRNPL-BGHLV23-HepG2.set23-excluded-upon-knockdown')


NO SUFFICIENT POSITIVE OR NEGATIVE SIGNIFICANT ANNOTATIONS:
256
275
279
444
460
586
645
681
697
722reseq2


Writing 193 tasks as an array-job.
Wrote commands to /projects/ps-yeolab3/bay001/maps/bash_scripts/7-25-2018/SE_IDR_PNGS.sh.


# RBPclasses

In [43]:
classes = pd.read_table(
    "/projects/ps-yeolab3/bay001/reference_data/ENCODE/RBPclasses_20180401.txt"
)
classes.head()

,Unnamed: 0,Class#,Class ID,SubmittedDueToFamily
0,SND1-HepG2,1,CDS,1
1,FXR1-K562,1,CDS,1
2,SND1-K562,1,CDS,1
3,FXR2-HepG2,1,CDS,1
4,G3BP1-HepG2,1,CDS,1


In [44]:
def get_rbp(row):
    return row['Unnamed: 0'].split('-')[0]
def get_cell(row):
    return row['Unnamed: 0'].split('-')[1]
classes['rbp'] = classes.apply(get_rbp, axis=1)
classes['cell'] = classes.apply(get_cell, axis=1)
classes.head()

,Unnamed: 0,Class#,Class ID,SubmittedDueToFamily,rbp,cell
0,SND1-HepG2,1,CDS,1,SND1,HepG2
1,FXR1-K562,1,CDS,1,FXR1,K562
2,SND1-K562,1,CDS,1,SND1,K562
3,FXR2-HepG2,1,CDS,1,FXR2,HepG2
4,G3BP1-HepG2,1,CDS,1,G3BP1,HepG2


In [45]:
def return_rbps_within_same_class(rbp, cell, df=classes):
    """ given an rbp and cell type, return list of other rbps in same binding class. """
    class_num = df[(df['cell']==cell) & (df['rbp']==rbp)]['Class#'].values[0]
    return list(set(df[(df['Class#']==class_num) & (df['cell']==cell)]['Unnamed: 0']))
          

def get_uid_from_rbp_and_cell(rbp, cell, merged=merged):
    """ given an rbp name and celltype, return uid """
    return merged[(merged['RBP']==rbp) & (merged['Cell line']==cell)]['uID'].values[0]


def get_groupid(rbp, cell, df=classes):
    class_num = df[(df['cell']==cell) & (df['rbp']==rbp)]['Class#'].values[0]
    return class_num

def compare_rbp_against_all_others_in_class(uid, df=classes, merged=merged):
    """ Given an rbp uID, return a list of uIDs in its same group. """
    all_uids_to_compare = []
    r1, r2, i, rbp, cell, jxc_se = get_clip_file_from_uid(uid, df=merged)
    rbps_list = return_rbps_within_same_class(rbp, cell, df)
    for protein in rbps_list:
        rbp_name, cell_type = protein.split('-')
        try:
            all_uids_to_compare.append(get_uid_from_rbp_and_cell(rbp_name, cell_type))
        except IndexError:
            # print(rbp_name, cell_type)
            pass
    if len(all_uids_to_compare) > 0:
        return all_uids_to_compare
    else:
        return None
    
get_groupid('RBFOX2','HepG2')

4

In [39]:
img_extension = ['png']
pos_splicing_suffix = '-included-upon-knockdown'
neg_splicing_suffix = '-excluded-upon-knockdown'

if not os.path.exists(bash_scripts_dir):
    ! mkdir $bash_scripts_dir

read_type = 'whole_read_xcompare'
normalization_levels = [4]

### Force override of maps
force = False

### DEFINE BACKGROUNDS (THESE ARE STATIC AND DON'T CHANGE) ###
k562_background_ce = os.path.join(control_annotation_dir, 'K562_constitutive_exons')
k562_background_nse_all = os.path.join(control_annotation_dir, 'K562_native_cassette_exons_all')
k562_background_nse_inc = os.path.join(control_annotation_dir, 'K562_natively_included_cassette_exons')
k562_background_nse_exc = os.path.join(control_annotation_dir, 'K562_natively_excluded_cassette_exons')
k562_background_nse_avg = os.path.join(control_annotation_dir, 'K562_native_cassette_exons_avg')

hepg2_background_ce = os.path.join(control_annotation_dir, 'HepG2_constitutive_exons')
hepg2_background_nse_all = os.path.join(control_annotation_dir, 'HepG2_native_cassette_exons_all')
hepg2_background_nse_inc = os.path.join(control_annotation_dir, 'HepG2_natively_included_cassette_exons')
hepg2_background_nse_exc = os.path.join(control_annotation_dir, 'HepG2_natively_excluded_cassette_exons')
hepg2_background_nse_avg = os.path.join(control_annotation_dir, 'HepG2_native_cassette_exons_avg')

# manifest for eric (clip id / rnaseq id / group # / pos / neg)
# manifest_file = '/projects/ps-yeolab3/bay001/maps/current/se_xcompare_manifest.txt'
manifest_file = '/projects/ps-yeolab3/bay001/reference_data/ENCODE/se_xcompare_manifest.nSE.txt'
o = open(manifest_file, 'w')
o.write("{}\t{}\t{}\t{}\t{}\n".format(
    "clip_id", "uID", "group_id", "pos_file_name", "neg_file_name"
))
print("number of entries: {}".format(len(set(merged['uID']))))
TOTAL_COMPARISONS = 0  # let's add up the total (expected) number of comparisons we're supposed to make.

# no_rnaseq = [] # uIDs for which we don't have rna seq expt ids for
no_rnaseq_yet = [] # uIDs for which we have an expt id, but haven't downloaded the data yet (or there were no significant splice events)
cmds = []
print("Number of UIDs: {}".format(len(merged['uID'])))
for normalization_level in normalization_levels:
    for ext in img_extension:
        for uid in set(merged['uID']):
            # Regardless of how many pos/neg events there are, generate the maps for each RBP with just the (nSE-All and nSE-Avg) annotations.
            r1, r2, i, rbp, cell, _ = get_clip_file_from_uid(uid)
            if cell == 'K562':
                background2 = k562_background_nse_all
                background5 = k562_background_nse_avg
            elif cell == 'HepG2':
                background2 = hepg2_background_nse_all
                background5 = hepg2_background_nse_avg
            else:
                print(cell)
            
            for r in [r1, r2]:
                name = os.path.basename(r).replace('.bam','.nSE.{}.{}.{}'.format(
                    cell, normalization_level, ext
                ))
                output_filename = os.path.join(
                    params[read_type]['output_dir'],
                    name
                )
                allmeans = os.path.join(
                    params[read_type]['output_dir'],
                    "{}*{}*native_cassette_exons_all.means.txt".format(uid, uid)
                )
                avgmeans = os.path.join(
                    params[read_type]['output_dir'],
                    "{}*{}*native_cassette_exons_avg.means.txt".format(uid, uid)
                )
                # Build the cmd line
                
                cmd = "python " + density_runner
                cmd = cmd + " --event {}".format('se')
                cmd = cmd + " --ipbam {}".format(r)
                cmd = cmd + " --inputbam {}".format(i)
                cmd = cmd + " --output {}".format(output_filename)
                cmd = cmd + " --annotations {} {}".format( 
                    background2, background5,
                )
                cmd = cmd + " --annotation_type {} {}".format( 
                    'eric', 'eric',
                )
                cmd = cmd + " --normalization_level {}".format(normalization_level)
                o.write("{}\t{}\t{}\t{}\t{}\n".format(
                    os.path.basename(r),
                    uid,
                    get_groupid(rbp, cell),
                    "-",
                    "-",
                ))
                if not os.path.exists(output_filename) or force == True:
                    cmds.append(cmd)
                
            other_uids_to_compare = compare_rbp_against_all_others_in_class(uid)
            if other_uids_to_compare is not None:
                # Just a check to make sure we're making proper comparisons...
                other_rbps = []
                for x in range(min(5, len(other_uids_to_compare))):
                    try:
                        _, _, _, other_rbp, other_cell, _ = get_clip_file_from_uid(other_uids_to_compare[x])
                        other_rbps.append("{}-{}".format(other_rbp, other_cell))
                    except Exception as e:
                        print(len(other_uids_to_compare))
                print("comparing {}-{} to {}..".format(rbp, cell, other_rbps))
                
                # end check
                
                for other_uid in other_uids_to_compare:
                    output_filename = ""
                    
                    TOTAL_COMPARISONS += 1
                    
                    _, _, _, _, _, jxc_se = get_clip_file_from_uid(other_uid)
                    
                    ##### get the positive and negative associated annotations using this prefix #####
                    positive, negative = get_annotations_from_jxc_se(
                        jxc_se
                    )
                    ### we HAVE to have at both positive and negative annotations to plot ###
                    if(positive == None and negative == None):
                        # manifest for eric (clip id / rnaseq id / group # / pos / neg)
                        o.write("{}\t{}\t{}\t{}\t{}\n".format(
                            os.path.basename(r),
                            jxc_se,
                            get_groupid(rbp, cell),
                            "-",
                            "-",
                        ))
                    elif positive is None:
                        # manifest for eric (clip id / rnaseq id / group # / pos / neg)
                        o.write("{}\t{}\t{}\t{}\t{}\n".format(
                            os.path.basename(r),
                            jxc_se,
                            get_groupid(rbp, cell),
                            "-",
                            "-",
                        ))
                    elif negative is None:
                        # manifest for eric (clip id / rnaseq id / group # / pos / neg)
                        o.write("{}\t{}\t{}\t{}\t{}\n".format(
                            os.path.basename(r),
                            jxc_se,
                            get_groupid(rbp, cell),
                            "-",
                            "-",
                        ))
                    else:
                        ### uses RBP name to ensure positive and negative annotations are being pulled ###
                        pos_prefix = os.path.basename(positive).split('-')[0]
                        neg_prefix = os.path.basename(negative).split('-')[0]
                        posdf = pd.read_table(positive)
                        negdf = pd.read_table(negative)

                        ### Foreach replicate, build teh command used to call the python script.
                        for r in [r1, r2]:
                            
                            # If there are not enough positive events and not enough negative events: write to file 
                            if ((posdf.shape[0] < 100) & (negdf.shape[0] < 100)):
                                # manifest for eric (clip id / rnaseq id / group # / pos / neg)
                                o.write("{}\t{}\t{}\t{}\t{}\n".format(
                                    os.path.basename(r),
                                    os.path.basename(positive.split('.')[0]),
                                    get_groupid(rbp, cell),
                                    "-",
                                    "-",
                                ))
                            # Else if we have enough positive events, write build the commandline and write positive to file.
                            elif ((posdf.shape[0] >= 100) & (negdf.shape[0] < 100)):
                                # Build the cmd line
                                cmd = "module load rbpmaps;python " + density_runner
                                cmd = cmd + " --event {}".format('se')
                                cmd = cmd + " --ipbam {}".format(r)
                                cmd = cmd + " --inputbam {}".format(i)
                                cmd = cmd + " --output {}".format(output_filename)
                                cmd = cmd + " --annotations {} {}".format( # {} {} {} {} {}".format(
                                    positive, background1, # background2, background3, background4, background5
                                )
                                cmd = cmd + " --annotation_type {} {}".format( # {} {} {} {} {}".format(
                                    'rmats', 'eric', # , 'eric', 'eric', 'eric', 'eric', 'eric' 
                                )
                                # cmd = cmd + " --chrom_sizes {}".format(chrom_sizes)
                                # cmd = cmd + " --testnum {} {}".format(0, 1)
                                # cmd = cmd + " --bgnum {}".format(3) # test against native SE
                                cmd = cmd + " --normalization_level {}".format(normalization_level)
                                if not os.path.exists(output_filename) or force == True:
                                    cmds.append(cmd)
                                # manifest for eric (clip id / rnaseq id / group # / pos / neg)
                                o.write("{}\t{}\t{}\t{}\t{}\n".format(
                                    os.path.basename(r),
                                    os.path.basename(positive.split('.')[0]),
                                    get_groupid(rbp, cell),
                                    output_filename.replace('.{}'.format(ext), '.{}.means.txt'.format(os.path.basename(positive))),
                                    "-"
                                ))
                            # Else if we have enough negative events but not positive, build commandline with just negative
                            elif ((negdf.shape[0] >= 100) & (posdf.shape[0] < 100)):
                                # Build the cmd line
                                cmd = "module load rbpmaps;python " + density_runner
                                cmd = cmd + " --event {}".format('se')
                                cmd = cmd + " --ipbam {}".format(r)
                                cmd = cmd + " --inputbam {}".format(i)
                                cmd = cmd + " --output {}".format(output_filename)
                                cmd = cmd + " --annotations {} {}".format( # {} {} {} {} {}".format(
                                    negative, background1 # , background2, background3, background4, background5
                                )
                                cmd = cmd + " --annotation_type {} {}".format( # {} {} {} {} {}".format(
                                    'rmats', 'eric',# , 'eric', 'eric', 'eric', 'eric', 'eric' 
                                )
                                # cmd = cmd + " --chrom_sizes {}".format(chrom_sizes)
                                # cmd = cmd + " --testnum {} {}".format(0, 1)
                                # cmd = cmd + " --bgnum {}".format(3) # test against native SE
                                cmd = cmd + " --normalization_level {}".format(normalization_level)
                                if not os.path.exists(output_filename) or force == True:
                                    cmds.append(cmd)
                                # manifest for eric (clip id / rnaseq id / group # / pos / neg)
                                o.write("{}\t{}\t{}\t{}\t{}\n".format(
                                    os.path.basename(r),
                                    os.path.basename(positive.split('.')[0]),
                                    get_groupid(rbp, cell),
                                    "-",
                                    output_filename.replace('.{}'.format(ext), '.{}.means.txt'.format(os.path.basename(negative))),
                                ))
                            # Else if we have enough events for both, generate commandline using both
                            elif ((negdf.shape[0] >= 100) & (posdf.shape[0] >= 100)):
                                # Build the cmd line
                                cmd = "module load rbpmaps;python " + density_runner
                                cmd = cmd + " --event {}".format('se')
                                cmd = cmd + " --ipbam {}".format(r)
                                cmd = cmd + " --inputbam {}".format(i)
                                cmd = cmd + " --output {}".format(output_filename)
                                cmd = cmd + " --annotations {} {}".format( # {} {} {} {} {}".format(
                                    positive, negative# , background1, background2, background3, background4, background5
                                )
                                cmd = cmd + " --annotation_type {} {}".format( # {} {} {} {} {}".format(
                                    'rmats', 'rmats',# , 'eric', 'eric', 'eric', 'eric', 'eric' 
                                )
                                # cmd = cmd + " --chrom_sizes {}".format(chrom_sizes)
                                # cmd = cmd + " --testnum {} {}".format(0, 1)
                                # cmd = cmd + " --bgnum {}".format(3) # test against native SE
                                cmd = cmd + " --normalization_level {}".format(normalization_level)
                                if not os.path.exists(output_filename) or force == True:
                                    cmds.append(cmd)
                                # manifest for eric (clip id / rnaseq id / group # / pos / neg)
                                o.write("{}\t{}\t{}\t{}\t{}\n".format(
                                    os.path.basename(r),
                                    os.path.basename(positive.split('.')[0]),
                                    get_groupid(rbp, cell),
                                    output_filename.replace('.{}'.format(ext), '.{}.means.txt'.format(os.path.basename(positive))),
                                    output_filename.replace('.{}'.format(ext), '.{}.means.txt'.format(os.path.basename(negative))),
                                ))
            
o.close()
bash_script_sh = '/projects/ps-yeolab3/bay001/maps/bash_scripts/{}/{}-SE_XCOMPARE2_{}.{}.sh'.format(
    current_date, 
    params[read_type]['prefix'], 
    ext,
    normalization_level
)

Submitter(
    cmds, 
    "{}-SE_XCOMPARE2_{}".format(params[read_type]['prefix'], ext), 
    sh=bash_script_sh,
    submit=True,
    array=True,
    walltime='3:00:00',
    queue='home-yeo'
)

with open(bash_script_sh + ".slow.sh", 'w') as o:
    o.write("#!/usr/bin/env bash\n")
    for cmd in cmds:
        o.write(cmd + "\n")

### Print any missing/unavailable annotations to check over ###
print("uIDs for which we have an RNASEQ ID, but we don't have the splicing data yet: {}".format(
        len(no_rnaseq_yet))
     )

number of entries: 203
Number of UIDs: 203
comparing SRSF9-HepG2 to ['U2AF1-HepG2', 'SF3A3-HepG2', 'U2AF2-HepG2', 'NCBP2-HepG2', 'RBM5-HepG2']..
comparing TIA1-HepG2 to ['GRSF1-HepG2', 'PCBP1-HepG2', 'TIAL1-HepG2', 'AKAP1-HepG2', 'LSM11-HepG2']..
comparing IGF2BP3-HepG2 to ['GRSF1-HepG2', 'PCBP1-HepG2', 'TIAL1-HepG2', 'AKAP1-HepG2', 'LSM11-HepG2']..
comparing SUPV3L1-K562 to ['WDR43-K562', 'AARS-K562', 'SUPV3L1-K562', 'ZC3H8-K562', 'TROVE2-K562']..
comparing TRA2A-HepG2 to ['GRWD1-HepG2', 'PRPF8-HepG2', 'RBM15-HepG2', 'BCLAF1-HepG2', 'PPIG-HepG2']..
comparing RBM22-HepG2 to ['EFTUD2-HepG2', 'SUPV3L1-HepG2', 'RBM22-HepG2']..
comparing PPIG-HepG2 to ['GRWD1-HepG2', 'PRPF8-HepG2', 'RBM15-HepG2', 'BCLAF1-HepG2', 'PPIG-HepG2']..
comparing BUD13-HepG2 to ['GRWD1-HepG2', 'PRPF8-HepG2', 'RBM15-HepG2', 'BCLAF1-HepG2', 'PPIG-HepG2']..
comparing EXOSC5-K562 to ['HNRNPU-K562', 'EIF4G2-K562', 'SMNDC1-K562', 'HNRNPUL1-K562', 'AGGF1-K562']..
comparing DDX3X-HepG2 to ['U2AF1-HepG2', 'SF3A3-HepG2', 'U2

comparing LSM11-K562 to ['HNRNPU-K562', 'EIF4G2-K562', 'SMNDC1-K562', 'HNRNPUL1-K562', 'AGGF1-K562']..
comparing XRN2-K562 to ['HNRNPU-K562', 'EIF4G2-K562', 'SMNDC1-K562', 'HNRNPUL1-K562', 'AGGF1-K562']..
comparing U2AF1-K562 to ['METAP2-K562', 'ZNF800-K562', 'FMR1-K562', 'SRSF1-K562', 'U2AF2-K562']..
comparing HNRNPA1-K562 to ['HNRNPU-K562', 'EIF4G2-K562', 'SMNDC1-K562', 'HNRNPUL1-K562', 'AGGF1-K562']..
comparing GTF2F1-HepG2 to ['AGGF1-HepG2', 'DDX52-HepG2', 'SUGP2-HepG2', 'KHSRP-HepG2', 'CDC40-HepG2']..
comparing TRA2A-K562 to ['METAP2-K562', 'ZNF800-K562', 'FMR1-K562', 'SRSF1-K562', 'U2AF2-K562']..
comparing LARP7-K562 to ['WDR43-K562', 'AARS-K562', 'SUPV3L1-K562', 'ZC3H8-K562', 'TROVE2-K562']..
comparing LIN28B-HepG2 to ['GRWD1-HepG2', 'PRPF8-HepG2', 'RBM15-HepG2', 'BCLAF1-HepG2', 'PPIG-HepG2']..
comparing TAF15-HepG2 to ['AGGF1-HepG2', 'DDX52-HepG2', 'SUGP2-HepG2', 'KHSRP-HepG2', 'CDC40-HepG2']..
comparing ZRANB2-K562 to ['HNRNPU-K562', 'EIF4G2-K562', 'SMNDC1-K562', 'HNRNPUL1-K56

comparing YBX3-K562 to ['METAP2-K562', 'ZNF800-K562', 'FMR1-K562', 'SRSF1-K562', 'U2AF2-K562']..
comparing DDX55-K562 to ['TIA1-K562', 'CPSF6-K562', 'LARP4-K562', 'IGF2BP2-K562', 'APOBEC3C-K562']..
comparing NOLC1-HepG2 to ['SMNDC1-HepG2', 'NIP7-HepG2', 'SSB-HepG2', 'TROVE2-HepG2', 'NOLC1-HepG2']..
comparing RBM15-HepG2 to ['GRWD1-HepG2', 'PRPF8-HepG2', 'RBM15-HepG2', 'BCLAF1-HepG2', 'PPIG-HepG2']..
comparing EXOSC5-HepG2 to ['AGGF1-HepG2', 'DDX52-HepG2', 'SUGP2-HepG2', 'KHSRP-HepG2', 'CDC40-HepG2']..
comparing FMR1-K562 to ['METAP2-K562', 'ZNF800-K562', 'FMR1-K562', 'SRSF1-K562', 'U2AF2-K562']..
comparing FXR2-K562 to ['METAP2-K562', 'ZNF800-K562', 'FMR1-K562', 'SRSF1-K562', 'U2AF2-K562']..
comparing BCCIP-HepG2 to ['AGGF1-HepG2', 'DDX52-HepG2', 'SUGP2-HepG2', 'KHSRP-HepG2', 'CDC40-HepG2']..
comparing HNRNPL-K562 to ['HNRNPU-K562', 'EIF4G2-K562', 'SMNDC1-K562', 'HNRNPUL1-K562', 'AGGF1-K562']..
comparing HNRNPUL1-HepG2 to ['AGGF1-HepG2', 'DDX52-HepG2', 'SUGP2-HepG2', 'KHSRP-HepG2', 'CD

Writing 500 tasks as an array-job.
Wrote commands to /projects/ps-yeolab3/bay001/maps/bash_scripts/8-20-2018/whole_read_xcompare-SE_XCOMPARE2_png.41.sh.
Submitted script to queue home-yeo.
 Job ID: 12409950
Writing 435 tasks as an array-job.
Wrote commands to /projects/ps-yeolab3/bay001/maps/bash_scripts/8-20-2018/whole_read_xcompare-SE_XCOMPARE2_png.42.sh.


uIDs for which we have an RNASEQ ID, but we don't have the splicing data yet: 362


Submitted script to queue home-yeo.
 Job ID: 12409951


# Let's check to make sure we've made all se splice maps

In [6]:
# this is the total number of maps we've made. This includes both replicates, and only contains maps for expts which have at least 1 significant positive and negative event
num_maps = len(glob.glob(os.path.join(params['whole_read']['output_dir'], "*.png")))

print("{} corresponds to {} RBPs with both eCLIP and RMATS significant events".format(num_maps, num_maps/2))

386 corresponds to 193 RBPs with both eCLIP and RMATS significant events


In [7]:
# Ensure that we have the correct number of rmats files by re-filtering them independently:
orig_jxc_dir = '/projects/ps-yeolab3/encode/rnaseq/alt_splicing/graveley_rmats_current' # where we downloaded data from xintao
out_dir = '/home/bay001/scratch/encode/rmats_subset_test' # we don't need to keep any of these, just calculate the number
jxc_files = glob.glob(os.path.join(orig_jxc_dir, '*.SE.MATS.JunctionCountOnly.txt')) # all jxc SE files
print("Total number of downloaded files (should be 473): {}".format(len(jxc_files)))

def subset_jxc_only(in_file, out_file):
    """
    Calls my 'nonredundant' script to return nonoverlapping splice events.
    """
    scripts_dir = '/home/bay001/projects/codebase/bfx/pyscripts/rnaseq/'
    cmd = 'python {} '.format(
        os.path.join(scripts_dir, 'subset_rmats_junctioncountonly.py')
    )
    cmd += '-i {} '.format(in_file)
    cmd += '-o {} '.format(out_file)
    cmd += '-e {} '.format('se')
    ! $cmd
    
def filter_jxc_file(fn, fdr=0.1, pvalue=0.05, sep=0.05, out_dir=out_dir):
    """ filters a JunctionCountsOnly file and saves significant up/down/all SE splice events. """
    dfp_size = 0
    dfn_size = 0
    
    df = pd.read_table(fn)
    dfs = df[(df['PValue'] < pvalue) & (df['FDR'] < fdr)]
    dfs_output = os.path.join(out_dir, os.path.basename(fn) + '.significant.txt')
    if dfs.shape[0] > 0:
        dfs.to_csv(dfs_output, sep='\t', index=False)
    
    dfp = df[(df['PValue'] < pvalue) & (df['IncLevelDifference'] > sep) & (df['FDR'] < fdr)]
    dfp_output = os.path.join(out_dir, os.path.basename(fn) + '.positive.txt')
    dfp_output_nr = os.path.join(out_dir, os.path.basename(fn) + '.positive.nr.txt')
    if dfp.shape[0] > 0:
        dfp.to_csv(dfp_output, sep='\t', index=False)
        subset_jxc_only(dfp_output, dfp_output_nr)
        dfp_size = pd.read_table(dfp_output_nr).shape[0]
        
    dfn = df[(df['PValue'] < pvalue) & (df['IncLevelDifference'] < -(sep)) & (df['FDR'] < fdr)]
    dfn_output = os.path.join(out_dir, os.path.basename(fn) + '.negative.txt')
    dfn_output_nr = os.path.join(out_dir, os.path.basename(fn) + '.negative.nr.txt')
    if dfn.shape[0] > 0:
        dfn.to_csv(dfn_output, sep='\t', index=False)
        subset_jxc_only(dfn_output, dfn_output_nr)
        dfn_size = pd.read_table(dfn_output_nr).shape[0]
    
    return dfs.shape[0], dfp.shape[0], dfn.shape[0], dfp_size, dfn_size
    
def get_filtered_jxc_num(jxc_files):
    """
    Iterates over filter_jxc_file() and returns a dictionary of the number of 
    significantly up/down/all SE events.
    """
    jxc_dict = defaultdict(dict)
    progress = tnrange(len(jxc_files))
    for jxc_file in jxc_files:
        significant, positive, negative, positive_nr, negative_nr = filter_jxc_file(jxc_file)
        jxc_dict[os.path.basename(jxc_file)] = {'sig':significant,'pos':positive,'neg':negative, 'pos_nr':positive_nr, 'neg_nr':negative_nr}
        progress.update(1)
    return jxc_dict

Total number of downloaded files (should be 473): 473


In [8]:
jxc_dict = get_filtered_jxc_num(jxc_files)

HBox(children=(IntProgress(value=0, max=473), HTML(value=u'')))

In [13]:
# make a copy of the merged manifest so we can delete stuff without worrying about modifying original dataframe
merged2 = merged.copy(deep=True)  
# convert rbp+cell line into a single string that we can search/use as a unique key
merged2['code'] = merged2.apply(lambda x: '{}-{}'.format(x['RBP'], x['Cell line']), axis=1) 
# check manually if these numbers match what is being printed in the figure:
for fn, num_events in jxc_dict.iteritems(): 
    # first check to see if jxc file is needed at all (is in our "final" matrix)
    if fn in set(merged['SE_jxc_file']):
        cell = fn.split('-')[2].split('.')[0]
        # if there are 0 sig events for this file, exclude this rbp+cell from this study (remove the row)
        if jxc_dict[fn]['pos_nr'] == 0:
            rbp = fn.split('-')[0]
            
            code = '{}-{}'.format(rbp, cell)
            print('not enough positive events: {} {}'.format(code, fn))
            merged2 = merged2[merged2['code']!=code]
        elif jxc_dict[fn]['neg_nr'] == 0:
            rbp = fn.split('-')[0]
            code = '{}-{}'.format(rbp, cell)
            print('not enough negative events: {} {}'.format(code, fn))
            merged2 = merged2[merged2['code']!=code]

print("we have {} valid filtered+nonredundant JXC files.".format(merged2.shape[0]))

not enough negative events: GRWD1-K562 GRWD1-BGKLV21-K562.set21.SE.MATS.JunctionCountOnly.txt
not enough positive events: CSTF2T-K562 CSTF2T-BGKLV13-K562.set13.SE.MATS.JunctionCountOnly.txt
not enough positive events: AKAP1-K562 AKAP1-LV08-K562.set10.SE.MATS.JunctionCountOnly.txt
not enough positive events: EXOSC5-HepG2 EXOSC5-BGHcLV07-HepG2.set39_H.SE.MATS.JunctionCountOnly.txt
not enough positive events: LSM11-K562 LSM11-BGKLV21-K562.set21.SE.MATS.JunctionCountOnly.txt
not enough positive events: FASTKD2-K562 FASTKD2-BGKLV13-K562.set13.SE.MATS.JunctionCountOnly.txt
not enough positive events: SLTM-K562 SLTM-BGKLV13-K562.set13.SE.MATS.JunctionCountOnly.txt
not enough positive events: DDX52-K562 DDX52-BGKLV19-K562.set19.SE.MATS.JunctionCountOnly.txt
not enough positive events: FAM120A-K562 FAM120A-BGKLV19-K562.set19.SE.MATS.JunctionCountOnly.txt
not enough positive events: DDX51-K562 DDX51-BGKLV19-K562.set19.SE.MATS.JunctionCountOnly.txt
we have 193 valid filtered+nonredundant JXC file

# The numbers match up so far...
- 193 valid jxc files with eCLIP is 10 less than 203 total jxc + eCLIP.
- double check that the 10 samples truly don't have enough significant events to be mapped:

In [ ]:
print("number of invalid/insufficient samples: {}".format(
    len(set(merged['SE_jxc_file']).difference(set(merged2['SE_jxc_file']))))
)
list_of_insignificant_jxc_files = list(set(merged['SE_jxc_file']).difference(set(merged2['SE_jxc_file'])))
for fn in list_of_insignificant_jxc_files:
    fn = os.path.join(orig_jxc_dir, fn)
    significant, positive, negative, positive_nr, negative_nr = filter_jxc_file(fn)
    print("fn: {} sig: {} pos: {} pos_nr: {} neg: {} neg_nr: {}".format(os.path.basename(fn), significant, positive, positive_nr, negative, negative_nr))

In [ ]:
list_of_insignificant_jxc_files

# AKAP1 K562 has 9 significant events but only 7 significant negative events
- make sure the IncLevelDifference is responsible for this difference

In [ ]:
AKAP1_K562 = pd.read_table(os.path.join(orig_jxc_dir, 'AKAP1-LV08-K562.set10.SE.MATS.JunctionCountOnly.txt'))
print(
    "Number passing pvalue/fdr: {}".format(
        AKAP1_K562[(AKAP1_K562['FDR'] < 0.1) & (AKAP1_K562['PValue'] < 0.05)].shape[0]
    )
)
print(
    "Number of sig. pos: {}".format(
        AKAP1_K562[(AKAP1_K562['FDR'] < 0.1) & (AKAP1_K562['PValue'] < 0.05) & (AKAP1_K562['IncLevelDifference'] > 0.05)].shape[0]
    )
)
print(
    "Number of sig. neg: {}".format(
        AKAP1_K562[(AKAP1_K562['FDR'] < 0.1) & (AKAP1_K562['PValue'] < 0.05) & (AKAP1_K562['IncLevelDifference'] < -0.05)].shape[0]
)

# Check splice map manifest

In [20]:
df = pd.read_table(
    '/projects/ps-yeolab3/bay001/maps/current/se_xcompare_manifest.txt'
)
print(df.shape)
print(len(set(df['rnaseq_id'])))
df

(9722, 5)
193


,clip_id,rnaseq_id,group_id,pos_file_name,neg_file_name
0,216_01_SRSF9.merged.r2.bam,U2AF1-BGHLV30-HepG2,2,/projects/ps-yeolab3/bay001/maps/current/se_xcompare/216_01_SRSF9.merged.r2.compare.U2AF1-BGHLV30-HepG2.1.U2AF1-BGHLV30-HepG2.set30-included-upon-knockdown.means.txt,/projects/ps-yeolab3/bay001/maps/current/se_xcompare/216_01_SRSF9.merged.r2.compare.U2AF1-BGHLV30-HepG2.1.U2AF1-BGHLV30-HepG2.set30-excluded-upon-knockdown.means.txt
1,216_02_SRSF9.merged.r2.bam,U2AF1-BGHLV30-HepG2,2,/projects/ps-yeolab3/bay001/maps/current/se_xcompare/216_02_SRSF9.merged.r2.compare.U2AF1-BGHLV30-HepG2.1.U2AF1-BGHLV30-HepG2.set30-included-upon-knockdown.means.txt,/projects/ps-yeolab3/bay001/maps/current/se_xcompare/216_02_SRSF9.merged.r2.compare.U2AF1-BGHLV30-HepG2.1.U2AF1-BGHLV30-HepG2.set30-excluded-upon-knockdown.means.txt
2,216_01_SRSF9.merged.r2.bam,SF3A3-BGHLV33-HepG2,2,/projects/ps-yeolab3/bay001/maps/current/se_xcompare/216_01_SRSF9.merged.r2.compare.SF3A3-BGHLV33-HepG2.1.SF3A3-BGHLV33-HepG2.set33-included-upon-knockdown.means.txt,/projects/ps-yeolab3/bay001/maps/current/se_xcompare/216_01_SRSF9.merged.r2.compare.SF3A3-BGHLV33-HepG2.1.SF3A3-BGHLV33-HepG2.set33-excluded-upon-knockdown.means.txt
3,216_02_SRSF9.merged.r2.bam,SF3A3-BGHLV33-HepG2,2,/projects/ps-yeolab3/bay001/maps/current/se_xcompare/216_02_SRSF9.merged.r2.compare.SF3A3-BGHLV33-HepG2.1.SF3A3-BGHLV33-HepG2.set33-included-upon-knockdown.means.txt,/projects/ps-yeolab3/bay001/maps/current/se_xcompare/216_02_SRSF9.merged.r2.compare.SF3A3-BGHLV33-HepG2.1.SF3A3-BGHLV33-HepG2.set33-excluded-upon-knockdown.means.txt
4,216_01_SRSF9.merged.r2.bam,U2AF2-BGHLV26-HepG2,2,/projects/ps-yeolab3/bay001/maps/current/se_xcompare/216_01_SRSF9.merged.r2.compare.U2AF2-BGHLV26-HepG2.1.U2AF2-BGHLV26-HepG2.set26-included-upon-knockdown.means.txt,/projects/ps-yeolab3/bay001/maps/current/se_xcompare/216_01_SRSF9.merged.r2.compare.U2AF2-BGHLV26-HepG2.1.U2AF2-BGHLV26-HepG2.set26-excluded-upon-knockdown.means.txt
5,216_02_SRSF9.merged.r2.bam,U2AF2-BGHLV26-HepG2,2,/projects/ps-yeolab3/bay001/maps/current/se_xcompare/216_02_SRSF9.merged.r2.compare.U2AF2-BGHLV26-HepG2.1.U2AF2-BGHLV26-HepG2.set26-included-upon-knockdown.means.txt,/projects/ps-yeolab3/bay001/maps/current/se_xcompare/216_02_SRSF9.merged.r2.compare.U2AF2-BGHLV26-HepG2.1.U2AF2-BGHLV26-HepG2.set26-excluded-upon-knockdown.means.txt
6,216_01_SRSF9.merged.r2.bam,NCBP2-BGHLV12-HepG2,2,/projects/ps-yeolab3/bay001/maps/current/se_xcompare/216_01_SRSF9.merged.r2.compare.NCBP2-BGHLV12-HepG2.1.NCBP2-BGHLV12-HepG2.set12-included-upon-knockdown.means.txt,/projects/ps-yeolab3/bay001/maps/current/se_xcompare/216_01_SRSF9.merged.r2.compare.NCBP2-BGHLV12-HepG2.1.NCBP2-BGHLV12-HepG2.set12-excluded-upon-knockdown.means.txt
7,216_02_SRSF9.merged.r2.bam,NCBP2-BGHLV12-HepG2,2,/projects/ps-yeolab3/bay001/maps/current/se_xcompare/216_02_SRSF9.merged.r2.compare.NCBP2-BGHLV12-HepG2.1.NCBP2-BGHLV12-HepG2.set12-included-upon-knockdown.means.txt,/projects/ps-yeolab3/bay001/maps/current/se_xcompare/216_02_SRSF9.merged.r2.compare.NCBP2-BGHLV12-HepG2.1.NCBP2-BGHLV12-HepG2.set12-excluded-upon-knockdown.means.txt
8,216_01_SRSF9.merged.r2.bam,RBM5-BGHcLV05-HepG2,2,-,-
9,216_02_SRSF9.merged.r2.bam,RBM5-BGHcLV05-HepG2,2,-,-


In [31]:
df = pd.read_table(
    "/projects/ps-yeolab3/bay001/maps/current_annotations/se/HNRNPA1-BGKLV21-K562.set21.SE.MATS.JunctionCountOnly.txt"
)
print(df[(df['IncLevelDifference']>=0.05) & (df['PValue']<=0.05) & (df['FDR']<=0.1)].shape)
print(df[(df['IncLevelDifference']<=-0.05) & (df['PValue']<=0.05) & (df['FDR']<=0.1)].shape)

(165, 23)
(99, 23)


# Create list of nSE control mean values

In [17]:
all_hepg2_files = glob.glob("/projects/ps-yeolab3/bay001/maps/current/se_xcompare/*.1.HepG2_native_cassette_exons_all.means.txt")
all_k562_files = glob.glob("/projects/ps-yeolab3/bay001/maps/current/se_xcompare/*.1.K562_native_cassette_exons_all.means.txt")
print(len(all_hepg2_files), len(all_k562_files))

(1468, 1086)


In [20]:
all_kept_hepg2_files = []
all_kept_k562_files = []

for fn in all_hepg2_files:
    if '.compare' not in fn:
        all_kept_hepg2_files.append(fn)

for fn in all_k562_files:
    if '.compare' not in fn:
        all_kept_k562_files.append(fn)
        
print(len(all_kept_hepg2_files), len(all_kept_k562_files))

(406, 0)


In [19]:
all_kept_hepg2_files

['/projects/ps-yeolab3/bay001/maps/current/se_xcompare/589_01_DDX6.merged.r2.589.1.HepG2_native_cassette_exons_all.means.txt',
 '/projects/ps-yeolab3/bay001/maps/current/se_xcompare/218_01_TRA2A.merged.r2.218.1.HepG2_native_cassette_exons_all.means.txt',
 '/projects/ps-yeolab3/bay001/maps/current/se_xcompare/275_01_CSTF2T.merged.r2.275.1.HepG2_native_cassette_exons_all.means.txt',
 '/projects/ps-yeolab3/bay001/maps/current/se_xcompare/256_01_SLTM.merged.r2.256.1.HepG2_native_cassette_exons_all.means.txt',
 '/projects/ps-yeolab3/bay001/maps/current/se_xcompare/681_01_DDX52.merged.r2.681.1.HepG2_native_cassette_exons_all.means.txt',
 '/projects/ps-yeolab3/bay001/maps/current/se_xcompare/206_02_HNRNPK.merged.r2.206.1.HepG2_native_cassette_exons_all.means.txt',
 '/projects/ps-yeolab3/bay001/maps/current/se_xcompare/315_02_FXR1.merged.r2.315.1.HepG2_native_cassette_exons_all.means.txt',
 '/projects/ps-yeolab3/bay001/maps/current/se_xcompare/676_02_RBFOX2.merged.r2.676.1.HepG2_native_cassett

# Compare values in scratch vs permanent storage (yeolab3)

In [13]:
all_excluded = glob.glob('/projects/ps-yeolab3/bay001/maps/current/se_idr_peak/*excluded-upon-knockdown.hist.txt')
print(len(all_excluded))
for excluded in all_excluded:
    permanent = glob.glob(os.path.join('/home/bay001/scratch/maps/se/idr/', os.path.basename(excluded)))
    if len(permanent) != 1:
        print(excluded)
    else:
        permanent = permanent[0]
        ! diff $permanent $excluded | head

174


In [14]:
all_included = glob.glob('/projects/ps-yeolab3/bay001/maps/current/se_idr_peak/*included-upon-knockdown.hist.txt')
print(len(all_excluded))
for included in all_included:
    permanent = glob.glob(os.path.join('/home/bay001/scratch/maps/se/idr/', os.path.basename(included)))
    if len(permanent) != 1:
        print(included)
    else:
        permanent = permanent[0]
        ! diff $permanent $included | head

174


In [18]:
all_excluded = glob.glob('/projects/ps-yeolab3/bay001/maps/current/se_peak/*excluded-upon-knockdown.hist.txt')
print(len(all_excluded))
for excluded in all_excluded:
    permanent = glob.glob(os.path.join('/home/bay001/projects/brian_rbpmaps_20180202/temporary_data/peak_bb/', os.path.basename(excluded)))
    if len(permanent) != 1:
        print(excluded)
    else:
        permanent = permanent[0]
        ! diff $permanent $excluded | head

0


In [ ]:
all_excluded = glob.glob('/projects/ps-yeolab3/bay001/maps/current/se_peak/*excluded-upon-knockdown.hist.txt')
print(len(all_excluded))
for excluded in all_excluded:
    permanent = glob.glob(os.path.join('/home/bay001/projects/brian_rbpmaps_20180202/temporary_data/peak_bb/', os.path.basename(excluded)))
    if len(permanent) != 1:
        print(excluded)
    else:
        permanent = permanent[0]
        ! diff $permanent $excluded | head